# Players Comparison

This notebook allows you to compare statistics between two NHL players.

## Get players stats

In [19]:
#! /home/db/.pyenv/shims/python

import requests
import pandas as pd
from ipywidgets import widgets, VBox, HBox, interactive_output
from IPython.display import display, HTML
import json

# Base server URL to get nhl stats
BASE_URL = "http://localhost:8080"

# player_a_name_search_pattern = "Connor McDavid"
# player_b_name_search_pattern = "Doughty"

player_a_name_search_pattern = widgets.Text(
    description='PLayer A search pattern:',
    style={'description_width': '150px'}
)

player_b_name_search_pattern = widgets.Text(
    description='PLayer B search pattern:',
    style={'description_width': '150px'}
)

search_players_button = widgets.Button(
    description='Search players',
    button_style='success'
)

def get_player_data(player_id):
    """Get player landing data by id

    Args:
        player_id (int)

    Returns:
        _type_: json-encoded content with player landing info
    """
    if player_id:
        response_stats = requests.get(f"{BASE_URL}/player/{player_id}/stats")
        if response_stats.status_code == 200:
            return response_stats.json()

def get_player_info(player_search_pattern):
    """Get player id by a search pattern

    Args:
        player_search_pattern (str): Player name to search for

    Returns:
        _type_: (int, str)
    """
    if player_search_pattern:
        response = requests.get(f"{BASE_URL}/player", params={"q": player_search_pattern})
        if response.status_code == 200:
            result = response.json()
            if isinstance(result, list) and len(result) > 0:
                player_id = result[0]['playerId']
                player_name = result[0]['name']
                other_possible_options = [f"\n{p['name']} (ID: {p['playerId']})" for p in result[1:]]
                print(f"✓ Found Player: {result[0]['name']} (ID: {player_id})")
                if other_possible_options:
                    print(f"Other possible players: {', '.join(other_possible_options)}")
                return (player_id, player_name)
            else:
                print(f"✗ Player not found")
        else:
            print(f"✗ Error searching Player {player_search_pattern}: {response.status_code}")

players_search_output = widgets.Output()

def search_players(_):
    global player_a_name, player_b_name, player_a_data, player_b_data
    with players_search_output:
        try:
            players_search_output.clear_output()
            player_a_info = get_player_info(player_a_name_search_pattern.value)
            player_b_info = get_player_info(player_b_name_search_pattern.value)
            
            if player_a_info and player_b_info:
                player_a_id, player_a_name = player_a_info
                player_b_id, player_b_name = player_b_info
                
                print(f"player_a_id={player_a_id}; player_b_id={player_b_id}")
                
                player_a_data = get_player_data(player_a_id)
                player_b_data = get_player_data(player_b_id)
        except Exception as e:
            print(f"Error: {str(e)}")

# search_players_button.on_click(search_players, remove=True)
search_players_button._click_handlers.callbacks.clear()
search_players_button.on_click(search_players)

display(
    VBox([
        HBox([player_a_name_search_pattern, player_b_name_search_pattern]),
        search_players_button,
        players_search_output,
    ])
)

## Season Totals Comparison

In [24]:
# Initialize player data variables if not already defined
if 'player_a_data' not in locals():
    player_a_data = None
if 'player_b_data' not in locals():
    player_b_data = None
if 'player_a_name' not in locals():
    player_a_name = "Player A"
if 'player_b_name' not in locals():
    player_b_name = "Player B"

season = widgets.Dropdown(
    options=[],
    description='Season:',
    style={'description_width': '150px'}
)

game_type_id = widgets.Dropdown(
    options=[("Regular Season", 2), ("Playoffs", 3)],
    value=2,
    description='Game Type:',
    style={'description_width': '150px'}
)

display_stats_by_season_button = widgets.Button(
    description='Stats by season',
    button_style='success',
    style={'description_width': '300px'}
)

display_agg_season_totals_button = widgets.Button(
    description='For All Seasons',
    button_style='success',
    style={'description_width': '300px'}
)

def update_season_dropdowns():
    """
    Update season dropdowns by iterating over both player season totals;
    Then merge unique seasons into a single list for the dropdown options;
    Then sort the seasons and select the most recent season by default.
    """
    seasons = set()
    if player_a_data and 'seasonTotals' in player_a_data:
        seasons.update([s['season'] for s in player_a_data['seasonTotals'] if s['season']])
    if player_b_data and 'seasonTotals' in player_b_data:
        seasons.update([s['season'] for s in player_b_data['seasonTotals'] if s['season']])
    
    sorted_seasons = sorted(seasons)
    season.options = [(str(s), s) for s in sorted_seasons]
    if sorted_seasons:
        # Select most recent season
        season.value = sorted_seasons[-1]
        
def highlight_larger(row):
    styles = [''] * len(row)
    try:
        a_idx = row.index.get_loc(player_a_name)
        b_idx = row.index.get_loc(player_b_name)
        a = pd.to_numeric(row[player_a_name], errors='coerce')
        b = pd.to_numeric(row[player_b_name], errors='coerce')
        if pd.notna(a) and pd.notna(b):
            if a > b:
                styles[a_idx] = 'background-color: red; color: white'
            elif b > a:
                styles[b_idx] = 'background-color: red; color: white'
    except Exception:
        pass
    return styles

def stats_by_season():
    def get_season_stats(player_data, season, game_type_id):
        if not player_data or 'seasonTotals' not in player_data:
            return None
    
        # Aggregate stats if multiple entries match
        matching_stats = [s for s in player_data['seasonTotals'] 
                        if s['season'] == season and s['gameTypeId'] == game_type_id and s['leagueAbbrev'] == 'NHL']
        
        if not matching_stats:
            return None
        
        if len(matching_stats) == 1:
            return matching_stats[0]
        
        # Aggregate multiple matching stats
        aggregated = {}
        numeric_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'pim', 
                        'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots']
        
        for field in numeric_fields:
            aggregated[field] = sum(s.get(field, 0) for s in matching_stats)
        
        return aggregated
            # return None

    def display_season_totals():
        if not player_a_data and not player_b_data:
            print("Please search for players first")
            return
        
        player_a_season_totals = get_season_stats(player_a_data, season.value, game_type_id.value)
        player_b_season_totals = get_season_stats(player_b_data, season.value, game_type_id.value)
        
        # Create comparison table
        comparison_data = []
        
        if player_a_season_totals or player_b_season_totals:
            stat_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'pim', 
                        'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots']
            
            for field in stat_fields:
                row = {'Metric': field}
                if player_a_season_totals:
                    row[f"{player_a_name}"] = player_a_season_totals.get(field, '-')
                if player_b_season_totals:
                    row[f"{player_b_name}"] = player_b_season_totals.get(field, '-')
                comparison_data.append(row)

            # Build DataFrame and highlight the larger value between the two players per row
            df_temp = pd.DataFrame(comparison_data)

            styled = df_temp.style.apply(highlight_larger, axis=1)
            display(styled)
        else:
            print("No season stats found for the selected criteria")
    display_season_totals()

def display_career_totals():
    if not player_a_data and not player_b_data:
        print("Please search for players first.")
        return
    
    # Helper function to get career totals
    def get_career_totals(player_data):
        if not player_data or 'careerTotals' not in player_data:
            return None
        return player_data['careerTotals']
    
    career_a = get_career_totals(player_a_data)
    career_b = get_career_totals(player_b_data)
    
    if not career_a and not career_b:
        print("No career totals found.")
        return
    
    # Display Regular Season Career Totals
    print("\nREGULAR SEASON CAREER TOTALS")
    
    reg_season_data = []
    if career_a and 'regularSeason' in career_a:
        reg_a = career_a['regularSeason']
        stat_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'plusMinus', 'pim',
                      'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots', 'shootingPctg']
        
        for field in stat_fields:
            row = {'Metric': field}
            row[f"{player_a_name}"] = reg_a.get(field, '-')
            if career_b and 'regularSeason' in career_b:
                reg_b = career_b['regularSeason']
                row[f"{player_b_name}"] = reg_b.get(field, '-')
            reg_season_data.append(row)
    
    if reg_season_data:
        df_reg = pd.DataFrame(reg_season_data)
        styled = df_reg.style.apply(highlight_larger, axis=1)
        display(styled)
    
    # Display Playoffs Career Totals
    print("\nPLAYOFFS CAREER TOTALS")
    
    playoff_data = []
    if career_a and 'playoffs' in career_a:
        playoff_a = career_a['playoffs']
        stat_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'plusMinus', 'pim',
                      'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots', 'shootingPctg']
        
        for field in stat_fields:
            row = {'Metric': field}
            row[f"{player_a_name}"] = playoff_a.get(field, '-')
            if career_b and 'playoffs' in career_b:
                playoff_b = career_b['playoffs']
                row[f"{player_b_name}"] = playoff_b.get(field, '-')
            playoff_data.append(row)
    
    if playoff_data:
        df_playoff = pd.DataFrame(playoff_data)
        styled = df_playoff.style.apply(highlight_larger, axis=1)
        display(styled)

update_season_dropdowns()

season_output = widgets.Output()
agg_season_totals_output = widgets.Output()

def on_display_season_click(_):
    season_output.clear_output()
    with season_output:
        stats_by_season()
        
def on_display_agg_season_totals_click(b):
    agg_season_totals_output.clear_output()
    with agg_season_totals_output:
        display_career_totals()

display_stats_by_season_button.on_click(on_display_season_click)
display_agg_season_totals_button.on_click(on_display_agg_season_totals_click)

display(
    VBox([
        HBox([season, game_type_id]),
        display_stats_by_season_button,
        season_output,
        display_agg_season_totals_button,
        agg_season_totals_output
    ])
)